In [1]:
import numpy as np
import json

import matplotlib.pyplot as plt
from time import time
import os
import sys

cwd = os.getcwd()

basedir = os.path.abspath(os.path.join(cwd, os.pardir))
if basedir not in sys.path:
    sys.path.insert(0, basedir)


datadir = os.path.join(basedir, 'vessel_data/CSAD')
filepath = os.path.join(datadir, [file for file in os.listdir(datadir) if file.endswith('.json')][2])
print(filepath)

c:\Users\j-ehy\OneDrive - NTNU\NTNU\9 semester\ProsjektOppgave\wavemodel\Wave-Model\vessel_data/CSAD\vessel_json.json


In [2]:
def timeit(func):
    # Function for calculating execution time.
    def wrapper(*args, **kwargs):
        start = time()
        result = func(*args, **kwargs)
        end = time()
        print(f"Execution time of '{func.__name__}': {end - start}")
        return result
    return wrapper

In [3]:
# Get the parameters from CSAD

with open(filepath, 'r') as f:
    data = json.load(f)



Mrb = np.asarray(data['MRB'])
A = np.asarray(data['A'])
B = np.asarray(data['B'])
C = np.asarray(data['C'])
freqs = np.asarray(data['freqs'])
headings = np.asarray(data['headings'])
Bv = np.asarray(data['Bv'])
# Store RAOs as complex values -> magnitude as real value and phase as imaginary
forceRAO = np.asarray(data['forceRAO']['amp'])[:, :, :, 0] + np.asarray(data['forceRAO']['phase'])[:, :, :, 0]*1j
motionRAO = np.asarray(data['motionRAO']['amp'])[:, :, :, 0] + np.asarray(data['motionRAO']['phase'])[:, :, :, 0]*1j
QTFs = np.asarray(data['driftfrc']['amp'])[:, :, :, 0]
QTFs[0].shape

(47, 36)

In [4]:
from numpy import dtype


@timeit
def full_qtf(qtfs, headings, freqs, wave_freqs, dof=0):
    # Generate full QTF by Newman approximation
    num_headings = len(headings)
    num_freq = len(wave_freqs)
    freq_indices = [np.argmin(np.abs(freqs - w)) for w in wave_freqs]
    Q = np.zeros((num_headings, num_freq, num_freq))
    Qjk = qtfs[dof].copy()
    for i in range(num_headings):
        for j, freq_j in enumerate(freq_indices):
            Q_j = Qjk[freq_j, i]
            for k, freq_k in enumerate(freq_indices):
                Q_k = Qjk[freq_k, i]
                Q[i, j, k] = 0.5*(Q_j + Q_k)
    return Q

@timeit
def full_qtf_alt(qtfs, headings, freqs, wave_freqs, dof=0):
    num_headings = len(headings)
    num_freq = len(wave_freqs)
    freq_indices = [np.argmin(np.abs(freqs - w)) for w in wave_freqs]
    Q = np.zeros((num_headings, num_freq, num_freq))
    Qdiag = qtfs[dof, [freq_indices], :].copy()
    for i in range(len(headings)):
        Q[i] = 0.5*(Qdiag[0, :, i, None] + Qdiag[0, :, i])
    return Q

N = 50 # Number of wave frequencies
wave_freqs = np.linspace(0.1, 3.14, N, endpoint=False)
# wave_freqs = freqs

print(f"Result diff: {np.sum(full_qtf_alt(QTFs, headings, freqs, wave_freqs, dof=0) - full_qtf(QTFs, headings, freqs, wave_freqs, dof=0))}")

@timeit
def full_qtf_6dof():
    Qd_surge = full_qtf(QTFs, headings, freqs, wave_freqs, dof=0)
    Qd_sway = full_qtf(QTFs, headings, freqs, wave_freqs, dof=1)
    Qd_heave = full_qtf(QTFs, headings, freqs, wave_freqs, dof=2)
    Qd_roll = full_qtf(QTFs, headings, freqs, wave_freqs, dof=3)
    Qd_pitch = full_qtf(QTFs, headings, freqs, wave_freqs, dof=4)
    Qd_yaw = full_qtf(QTFs, headings, freqs, wave_freqs, dof=5)
    return np.vstack([[Qd_surge], [Qd_sway], [Qd_heave], [Qd_roll], [Qd_pitch], [Qd_yaw]])

Qd = full_qtf_6dof()

Execution time of 'full_qtf_alt': 0.0059854984283447266
Execution time of 'full_qtf': 0.06585049629211426
Result diff: 0.0
Execution time of 'full_qtf': 0.05286097526550293
Execution time of 'full_qtf': 0.05388498306274414
Execution time of 'full_qtf': 0.04983353614807129
Execution time of 'full_qtf': 0.04887247085571289
Execution time of 'full_qtf': 0.041925668716430664
Execution time of 'full_qtf': 0.04085111618041992
Execution time of 'full_qtf_6dof': 0.29321956634521484


In [5]:
np.diagonal(Qd[0, 0])

array([4.86661285e-05, 4.86661285e-05, 4.86661285e-05, 1.01960638e-04,
       3.89088820e-04, 7.17648219e-04, 1.26851816e-03, 2.07429047e-03,
       3.64012335e-03, 5.59009633e-03, 7.04724367e-03, 1.18026259e-02,
       1.57227635e-02, 2.14435362e-02, 3.01072378e-02, 4.37648608e-02,
       4.37648608e-02, 6.64250444e-02, 6.64250444e-02, 1.06324207e-01,
       1.06324207e-01, 1.82583235e-01, 1.82583235e-01, 3.45914483e-01,
       3.45914483e-01, 3.45914483e-01, 3.45914483e-01, 7.55869233e-01,
       7.55869233e-01, 7.55869233e-01, 7.55869233e-01, 2.06969239e+00,
       2.06969239e+00, 2.06969239e+00, 2.63168924e+00, 2.63168924e+00,
       4.12212272e+00, 4.12212272e+00, 6.84665024e+00, 6.84665024e+00,
       6.84665024e+00, 1.20650551e+01, 1.20650551e+01, 1.20650551e+01,
       2.28807072e+01, 2.28807072e+01, 2.28807072e+01, 4.64502815e+01,
       4.64502815e+01, 4.64502815e+01])

In [6]:
QTFs[0].shape

(47, 36)

In [7]:
Qd_surge_heading0_diag = QTFs[0, :, 0]

In [8]:
Qd_surge_heading0 = 0.5*(Qd_surge_heading0_diag[:, None] + Qd_surge_heading0_diag)

In [9]:
freqs

array([0.2094395, 0.2513274, 0.3141592, 0.330694 , 0.3490658, 0.3695991,
       0.3926991, 0.4053668, 0.418879 , 0.4333231, 0.4487989, 0.4654211,
       0.4833219, 0.5026549, 0.5235987, 0.5463639, 0.5711986, 0.5983986,
       0.6283185, 0.6613879, 0.6981317, 0.7391983, 0.7853981, 0.837758 ,
       0.8975978, 0.9666439, 1.047197 , 1.142397 , 1.256637 , 1.396263 ,
       1.570796 , 1.795196 , 2.094395 , 2.166616 , 2.302377 , 2.457249 ,
       2.633355 , 2.837934 , 3.07547  , 3.358196 , 3.695992 , 4.109343 ,
       4.630203 , 5.297796 , 6.196435 , 7.453363 , 9.363913 ])

In [10]:
from waves.wave_loads import WaveLoad

In [11]:
amps = np.linspace(0, 3, N)
phase = np.random.random(N)*2*np.pi



waveload = WaveLoad(amps, wave_freqs, phase, headings, 0)
waveload._full_qtf_6dof(headings, freqs, QTFs)

Execution time of _full_qtf_6dof: 0.0070


array([[[[ 4.86661285e-05,  4.86661285e-05,  4.86661285e-05, ...,
           2.32251651e+01,  2.32251651e+01,  2.32251651e+01],
         [ 4.86661285e-05,  4.86661285e-05,  4.86661285e-05, ...,
           2.32251651e+01,  2.32251651e+01,  2.32251651e+01],
         [ 4.86661285e-05,  4.86661285e-05,  4.86661285e-05, ...,
           2.32251651e+01,  2.32251651e+01,  2.32251651e+01],
         ...,
         [ 2.32251651e+01,  2.32251651e+01,  2.32251651e+01, ...,
           4.64502815e+01,  4.64502815e+01,  4.64502815e+01],
         [ 2.32251651e+01,  2.32251651e+01,  2.32251651e+01, ...,
           4.64502815e+01,  4.64502815e+01,  4.64502815e+01],
         [ 2.32251651e+01,  2.32251651e+01,  2.32251651e+01, ...,
           4.64502815e+01,  4.64502815e+01,  4.64502815e+01]],

        [[ 4.79426604e-05,  4.79426604e-05,  4.79426604e-05, ...,
           2.22254655e+01,  2.22254655e+01,  2.22254655e+01],
         [ 4.79426604e-05,  4.79426604e-05,  4.79426604e-05, ...,
           2.22254655e

In [12]:
# Implempentation in wave loads module corresponds to that done in this notebook
# However - the speed is much faster in the wave module function.
# about 50 times faster actually.
np.sum(full_qtf_6dof() - waveload._full_qtf_6dof(headings, freqs, QTFs))

Execution time of 'full_qtf': 0.06681990623474121
Execution time of 'full_qtf': 0.06582403182983398
Execution time of 'full_qtf': 0.06685161590576172
Execution time of 'full_qtf': 0.045848846435546875
Execution time of 'full_qtf': 0.053853750228881836
Execution time of 'full_qtf': 0.05189251899719238
Execution time of 'full_qtf_6dof': 0.35508227348327637
Execution time of _full_qtf_6dof: 0.0060


0.0

In [13]:
forceRAO.shape
# Assumption that we have 6 dof x 36 freqs x 36 headings set of RAO transfer functions.

(6, 47, 36)

In [14]:
test_head_indx = np.array([0, 0, 0])
test_freqs_indx = np.array([1, 2, 3])

H = np.zeros(len(test_head_indx)) # The transfer functions to be used
